Chapter 5



In [ ]:
%pip install ISLP

In [28]:
from ISLP import load_data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score


In [9]:
default = load_data('Default')

In [10]:
default.head()

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879


In [11]:
default.describe()

,balance,income
count,10000.000000,10000.000000
mean,835.374886,33516.981876
std,483.714985,13336.639563
min,0.000000,771.967729
25%,481.731105,21340.462903
50%,823.636973,34552.644802
75%,1166.308386,43807.729272
max,2654.322576,73554.233495


In [12]:
np.random.seed(1)

In [13]:
default['student01'] = np.where(default['student'] == 'Yes', 1, 0)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(default.drop(['default','student'], axis=1), default['default'], test_size=0.2)

In [16]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

LogisticRegression()

In [17]:
log_reg_pred = log_reg.predict(X_test)

In [18]:
log_reg.score(X_test, y_test)

0.974

In [19]:
X_train, X_test, y_train, y_test = train_test_split(default.drop(['default','student'], axis=1), default['default'], test_size=0.2)

In [20]:
posteriors = log_reg.predict_proba(X_test)

In [21]:
log_reg_pred = np.where(posteriors[:,1] > 0.5, 1, 0)

In [26]:
log_reg_pred_error = np.mean(log_reg_pred != y_test)


In [27]:
log_reg_pred_error

1.0

In [30]:
errors = []
for seed in [1, 42, 123]:
    X_train, X_val, y_train, y_val = train_test_split(default.drop(['default','student'], axis=1), default['default'], test_size=0.3, random_state=seed)
    log_reg.fit(X_train, y_train)
    y_pred_prob = log_reg.predict_proba(X_val)[:, 1]
    y_pred = (y_pred_prob > 0.5).astype(int)
    error = np.mean(y_pred != y_val)
    errors.append(error)
    print(f"Validation Error (random_state={seed}): {error:.4f}")

print(f"\nAverage Validation Error: {np.mean(errors):.4f}")

Validation Error (random_state=1): 1.0000
Validation Error (random_state=42): 1.0000
Validation Error (random_state=123): 1.0000

Average Validation Error: 1.0000


6.


In [34]:
endog = (default["student01"] == "1").astype(int)
exog = sm.add_constant(default.drop(columns = ['default','student']))
glm_mod = sm.GLM(endog, exog, family = sm.families.Binomial())
glm_res = glm_mod.fit()
print(glm_res.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:              student01   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9996
Model Family:                Binomial   Df Model:                            3
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.4444e-09
Date:                Mon, 07 Jul 2025   Deviance:                   2.8910e-09
Time:                        00:39:28   Pearson chi2:                 1.44e-09
No. Iterations:                    28   Pseudo R-squ. (CS):                nan
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -29.5661   7.97e+04     -0.000      1.0

/usr/local/lib/python3.11/dist-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/usr/local/lib/python3.11/dist-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/usr/local/lib/python3.11/dist-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/usr/local/lib/python3.11/dist-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)


In [37]:
glm_res.params

,0
const,-2.956607e+01
balance,8.135918e-14
income,-1.687367e-15
student01,-3.719247e-15


In [42]:
def boot_fn(data, index):
    sample = data.iloc[index]
    X = (default["student01"] == "1").astype(int)
    X = sm.add_constant(X)
    glm_mod = sm.GLM(endog, exog, family = sm.families.Binomial())
    glm_res = glm_mod.fit()
    return glm_res.params[['income', 'balance']]

In [43]:
np.random.seed(1)
B = 1000
coefs = np.zeros((B, 2))

for b in range(B):
    sample_indices = np.random.choice(range(len(default.drop(columns = ['default','student']))), size=len(default.drop(columns = ['default','student'])), replace=True)
    coefs[b, :] = boot_fn(default.drop(columns = ['default','student']), sample_indices)

# Compute bootstrap standard errors
boot_se = coefs.std(axis=0)
print(f"Bootstrap SE (income):  {boot_se[0]:.6f}")
print(f"Bootstrap SE (balance): {boot_se[1]:.6f}")

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.11/dist-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/usr/local/lib/python3.11/dist-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/usr/local/lib/python3.11/dist-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/usr/local/lib/python3.11/dist-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  war

Bootstrap SE (income):  0.000000
Bootstrap SE (balance): 0.000000


/usr/local/lib/python3.11/dist-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/usr/local/lib/python3.11/dist-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/usr/local/lib/python3.11/dist-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/usr/local/lib/python3.11/dist-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)


In [45]:
boston = load_data('Boston')

medv = boston['medv']
n = len(medv)

In [47]:
mu_hat = np.mean(medv)
print(f"Estimate of population mean (mu^): {mu_hat:.4f}")

Estimate of population mean (mu^): 22.5328


In [48]:
std_sample = np.std(medv, ddof=1)
se_formula = std_sample / np.sqrt(n)

In [49]:
B = 1000
boot_means = []

for _ in range(B):
    sample = np.random.choice(medv, size=n, replace=True)
    boot_means.append(np.mean(sample))

se_bootstrap = np.std(boot_means)

In [50]:
ci_bootstrap = [mu_hat - 2 * se_bootstrap, mu_hat + 2 * se_bootstrap]
print(f"95% CI from bootstrap: {ci_bootstrap}")

95% CI from bootstrap: [21.750648216901766, 23.314964431319588]


In [51]:
ci_formula = [mu_hat - 2 * se_formula, mu_hat + 2 * se_formula]
print(f"95% CI from formula:   {ci_formula}")

95% CI from formula:   [21.715084029115605, 23.35052861910575]


In [53]:
mu_median = np.median(medv)
print(f"Median (mu^med): {mu_median:.4f}")

Median (mu^med): 21.2000


In [54]:
boot_medians = []

for _ in range(B):
    sample = np.random.choice(medv, size=n, replace=True)
    boot_medians.append(np.median(sample))

se_median = np.std(boot_medians)
print(f"Bootstrap SE of median: {se_median:.4f}")

Bootstrap SE of median: 0.3714


In [55]:
mu_10th = np.percentile(medv, 10)
print(f"10th percentile (mu^0.1): {mu_10th:.4f}")

10th percentile (mu^0.1): 12.7500


In [56]:
boot_p10 = []

for _ in range(B):
    sample = np.random.choice(medv, size=n, replace=True)
    boot_p10.append(np.percentile(sample, 10))

se_p10 = np.std(boot_p10)
print(f"Bootstrap SE of 10th percentile: {se_p10:.4f}")


Bootstrap SE of 10th percentile: 0.4970
